In [1]:
import numpy as np
import numba as nb

import ice_funcs as icef

## Todo:
- Increment global time with minimum time increment
- Continue function that updates values from the whole new value array thingy IDK
- Work on output of an array of ice positions that's timesliced according to global time

## Backburner-type-beat:
- make sure everything is numbafied
- Continue commenting in `ice_funcs.py`

## Improvements
- Monopole matching
- correct geometry factors $G_b$ and $H_b$
- Causal timings?



In [2]:
# CONSTANTS

L = 101
W = (L+1)//2
INITIAL_SAT = 1

In [3]:
gu = icef.GeneralUtilities(101) # general utilities
pu = icef.PhysicsUtilities(0.01, 10, X_0=1) # physics utilities
sru = icef.SaturationRelaxationUtilities(101, 200)

In [4]:
gru = icef.GrowthUtilities(101)

In [5]:
SnowflakeSim = icef.SnowflakeSimulation(101, 200)
# SnowflakeSim.GeneralU.neighbor_array

In [ ]:
neighbor_array_test = gu.neighbor_array
diffusion_rules_test = sru.diffusion_rules

print(diffusion_rules_test)

In [ ]:
default_ice_map = np.full((101, 101), False)
default_ice_map[-1,0] = True
default_ice_map[-2,0] = True
default_ice_map[-3,0] = True
default_ice_map[-3,1] = True

# %timeit test_boundary_map
boundary_map_test = gu.construct_boundary_map(default_ice_map)
print(boundary_map_test[-5:,:5])

sat_map_test = np.full((101,101), 1, dtype=np.float32)

normal_cells_test, boundary_cells_test = gu.distinguish_cells(default_ice_map, boundary_map_test)

opp_array_test = sru._construct_opp_array(boundary_cells_test, default_ice_map, neighbor_array_test)

In [ ]:
# pu.apply_boundary_condition(2, 2, sat_map_test, 0.5, boundary_map_test)

sat_map_test = np.full((101,101), 1, dtype=np.float32)

# for i in range(50):
#     sru._execute_relaxation_step(sat_map_test, normal_cells_test, boundary_cells_test, opp_array_test, neighbor_array_test, boundary_map_test)

new_thang = sru.diffuse_to_convergence(sat_map_test, 0.0001, normal_cells_test, boundary_cells_test, default_ice_map, boundary_map_test, neighbor_array_test, pu)
print(new_thang[1])
print(new_thang[2])
print(new_thang[0][-10:,:5])


%timeit sru.diffuse_to_convergence(sat_map_test, 0.001, normal_cells_test, boundary_cells_test, default_ice_map, boundary_map_test, neighbor_array_test, pu)

In [ ]:
gru.update_minimum_time(boundary_cells_test, sat_map_test, boundary_map_test, pu)
gru.minimum_time_increment

gru.update_filling_array(boundary_cells_test, sat_map_test, boundary_map_test, pu)
gru.filling_array[-5:,:5]